In [ ]:
%matplotlib inline

In [ ]:
from framed import load_odemodel, plot_timecourse, find_steady_state
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid') 

### Load a kinetic model of glycolysis/PPP in E. coli

In [ ]:
model = load_odemodel('models/Chassagnole2002_fixed.xml')

In [ ]:
Image('misc/chassagnole.png')

Let's quickly check the size of the model:

In [ ]:
print 'metabolites:', len(model.metabolites) 
print 'reactions:', len(model.reactions)  
print 'parameters:', len(model.get_parameters())

### Simulation - exercise 1

Let's run a time-course simulation

In [ ]:
ax = plot_timecourse(model, 100)

# this will avoid the legend to overlap with the plot
ax.get_legend().set_bbox_to_anchor(bbox=(1.4,1.2))

Nothing seems to happen. Why ?

### Simulation - exercise 2

Let's give it a glucose pulse and see what happens to some key metabolites: 

In [ ]:
model.concentrations['cglcex'] = 1

In [ ]:
plot_timecourse(model, 1, metabolites=['cglcex', 'cg6p', 'cfdp', 'cpep', 'cpyr'],
                xlabel='time', ylabel='concentration')
plt.title("1 second after pulse");

In [ ]:
plot_timecourse(model, 60, metabolites=['cglcex', 'cg6p', 'cfdp', 'cpep', 'cpyr'],
                xlabel='time', ylabel='concentration')
plt.title("1 minute after pulse");

In [ ]:
plot_timecourse(model, 1200, metabolites=['cglcex', 'cg6p', 'cfdp', 'cpep', 'cpyr'],
                xlabel='time', ylabel='concentration')
plt.title("20 min after pulse");

### Simulation - exercise 3

The kinetic model we are working with is simulating growth of E. coli in a chemostat. Let's check what is the pre-defined dilution rate.

In [ ]:
print 'Dilution rate (1/h):', model.constant_params['Dil'] * 3600

Let's run the same glucose-pulse simulation with a higher dilution rate (Dil = 0.2 $h^{-1}$).

In [ ]:
plot_timecourse(model, 1200, metabolites=['cglcex', 'cg6p', 'cfdp', 'cpep', 'cpyr'],
                xlabel='time', ylabel='concentration',
                parameters={'Dil': 0.2 / 3600})
plt.title("20 min after pulse");

Are the final metabolite concentrations the same ?

### Steady-state analysis

How does the dilution rate affect the metabolite concentrations and fluxes at steady-state? Let's find out!

In [ ]:
dilution_rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] # (1/h)

Calculate intracellular pyruvate concentration at different dilution rates.

In [ ]:
pyruvate = []

for dil in dilution_rates:
    x, v = find_steady_state(model, parameters={'Dil': dil / 3600})
    pyruvate.append(x['cpyr'])

In [ ]:
plt.plot(dilution_rates, pyruvate, 'o-')
plt.xlabel('Dilution rate (1/h)')
plt.ylabel('Pyruvate concentration (mM)')

How does the glucose uptake rate vary with dilution rate? 

In [ ]:
glc_uptake = []

for dil in dilution_rates:
    x, v = find_steady_state(model, parameters={'Dil': dil / 3600})
    glc_uptake.append(v['vPTS'])

In [ ]:
plt.plot(dilution_rates, glc_uptake, 'o-')
plt.xlabel('Dilution rate (1/h)')
plt.ylabel('Glucose uptake rate (mM/s)')

What about the ratio between glycolysis and the penthose-phosphate pathway?

In [ ]:
rates_pgi = []
rates_g6pdh = []

for dil in dilution_rates:
    x, v = find_steady_state(model, parameters={'Dil': dil / 3600})
    rates_pgi.append(v['vPGI'])
    rates_g6pdh.append(v['vG6PDH'])

In [ ]:
plt.plot(dilution_rates, rates_pgi, 'o-', dilution_rates, rates_g6pdh, 'o-')
plt.legend(['PGI', 'G6PDH'], loc='upper left')
plt.xlabel('Dilution rate (1/h)')
plt.ylabel('Reaction rate (mM/s)')